<a href="https://colab.research.google.com/github/FatemehNMT/MyROS/blob/main/2_DOF_Arm_Robot_and_Sensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2-DOF Arm Robot and Sensors:

1. Generate a based moving robot (base_link).
2. Set a 2-DOF arm on it(arm_joint1 and arm_joint2).
3. Put a sensor (camera or laser) at the end of the arm (camera_link).
4. Broadcast tf (tf broadcaster)
5. Define  and send out different frames (base_link, arm_joint1, arm_joint2, camera_link, object_frame).
6. Represent tf-tree applying rqt_tf_tree.
7. Apply tf listener to measure the distance of object's location relevant to the base or the camera.
8. Apply lookup_transform.
9. Present all the results in RViz with axes and markers.

Project Structure:

my_tf_demo/
├── urdf/
│   └── robot_with_arm.urdf.xacro
├── launch/
│   ├── display.launch
│   ├── gazebo.launch
│   └── tf_demo.launch
├── scripts/
│   ├── tf_broadcaster.py
│   └── tf_listener.py
└── CMakeLists.txt
└── package.xml

In [ ]:
cd simulation_ws/src/
catkin_create_pkg my_tf_demo roscpp rospy urdf sensor_msgs std_msgs
cd ..
catkin_make
source devel/setup.bash

In [ ]:
# if we are located in the simulation_ws folder
chmod +x src/my_tf_demo/scripts/tf_broadcaster.py
chmod +x src/my_tf_demo/scripts/tf_listener.py
chmod +x src/my_tf_demo/scripts/gazebo_object_tf.py

In [ ]:
catkin_make

In [ ]:
roslaunch my_tf_demo tf_demo.launch

In [ ]:
rosrun rqt_tf_tree rqt_tf_tree
# Savev Image